In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Load data and preprocess as before
wavelet_data = np.load('wavelet_transformed_data/layered_60x60_normalized.npy')
labels_df = pd.read_csv('labels/60x60.csv')
labels = labels_df['Label'].values

# Shuffle the data and labels together
indices = np.arange(wavelet_data.shape[0])
np.random.shuffle(indices)
wavelet_data_shuffled = wavelet_data[indices]
labels_shuffled = labels[indices]

# Convert labels to binary format
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_shuffled)
binary_labels = to_categorical(integer_encoded)

# Resize data to match EfficientNetB0 input shape, if necessary
data_resized = tf.image.resize(wavelet_data_shuffled, [224, 224]).numpy()

# Split the data
X_train, X_val, y_train, y_val = train_test_split(data_resized, binary_labels, test_size=0.1, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)

# Create the EfficientNet B0 base model with pretrained weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
base_model.trainable = False

# Create the top model for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global average pooling layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping with increased patience
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Revised learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.9  # Reduce lr by 10% after 5 epochs

lr_scheduler = LearningRateScheduler(scheduler)

# Adjust batch size if necessary
batch_size = 16  # Adjust based on your system's capability

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, shuffle=True, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_scheduler]
)


Epoch 1/20
67/67 [==============================] - 25s 259ms/step - loss: 0.2823 - accuracy: 0.8991 - val_loss: 0.5871 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - 16s 237ms/step - loss: 0.1474 - accuracy: 0.9462 - val_loss: 0.2504 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - 16s 232ms/step - loss: 0.1017 - accuracy: 0.9689 - val_loss: 0.3867 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 4/20
67/67 [==============================] - 16s 235ms/step - loss: 0.0826 - accuracy: 0.9708 - val_loss: 0.4698 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 5/20
67/67 [==============================] - 16s 236ms/step - loss: 0.0747 - accuracy: 0.9755 - val_loss: 1.1069 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 6/20
67/67 [==============================] - 16s 238ms/step - loss: 0.0587 - accuracy: 0.9745 - val_loss: 1.6803 - val_accuracy: 0.0500 - lr: 9.0000e-04
Epoch 7/20
67/67 [==============================] - 16s 232m

In [3]:
import os

# Define the directory to save the model
model_dir = 'models'

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model
model_save_path = 'models/60x60_trained_EfficientNet_model.h5'

# Save the model in HDF5 format
model.save(model_save_path, save_format='h5')

print(f"Model saved to {model_save_path}")

/home/lnagana/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to models/60x60_trained_EfficientNet_model.h5


In [5]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen.fit(X_train)  # Assuming X_train was used to fit the datagen during training
test_data_normalized = datagen.standardize(test_data_resized_np)

# Predict using the model
predictions = model.predict(test_data_normalized[:10])
# Assuming `predicted_classes` contains the predicted class indices from your model
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

print(predicted_labels)


1/1 [==============================] - 2s 2s/step
['Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal'
 'Normal' 'Normal']


In [6]:
individual_element = test_data_resized_np[178]
individual_element_batch = np.expand_dims(individual_element, axis=0)
individual_prediction = model.predict(individual_element_batch)
predicted_class_index = np.argmax(individual_prediction, axis=1) #class index
predicted_label = label_encoder.inverse_transform(predicted_class_index) #label

# Output
print("Predicted Class Index:", predicted_class_index[0])
print("Predicted Label:", predicted_label[0])


1/1 [==============================] - 0s 51ms/step
Predicted Class Index: 0
Predicted Label: Abnormal
